In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./all.csv", index_col=0)

In [3]:
df.head()

,kadencja,etap,klub,osoba
1,1,end,Akcja Polska,Antoni Macierewicz
2,1,end,Akcja Polska,Mariusz Marasek
3,1,end,Akcja Polska,Piotr Walerych
4,1,start,Bygdoska Lista Jedności Ludowej,Wojciech Mojzesowicz
5,1,start,Chrześcijańska Demokracja,Henryk Rospara


In [4]:
df_kluby_slowniczek = pd.read_csv("./slownik_klubow.csv", index_col=0)

In [5]:
kluby_dict = {v['old']: v['klub'] for k, v in df_kluby_slowniczek.iterrows()}

In [6]:
# nowe dane chyba nie wymagają czyszczenia (12.12.2015)
# df['klub'] = df['klub'].map(kluby_dict)

In [7]:
df.head()

,kadencja,etap,klub,osoba
1,1,end,Akcja Polska,Antoni Macierewicz
2,1,end,Akcja Polska,Mariusz Marasek
3,1,end,Akcja Polska,Piotr Walerych
4,1,start,Bygdoska Lista Jedności Ludowej,Wojciech Mojzesowicz
5,1,start,Chrześcijańska Demokracja,Henryk Rospara


In [8]:
# start i end wyglądają teraz OK
df[(df.etap=='end') & (df.kadencja==8)].klub.value_counts()

Prawo i Sprawiedliwość        234
Platforma Obywatelska         138
Kukiz’15                       41
Nowoczesna                     28
Polskie Stronnictwo Ludowe     16
Posłowie niezrzeszeni           3
dtype: int64

In [9]:
df["kadencja_ef"] = df["kadencja"] + (df["etap"] == "end")/2

In [10]:
# ostatnią kadencja jest początek 8
df = df.query("kadencja_ef <= 8")

In [11]:
przejscia = {}
for osoba, df_osoby in df.groupby('osoba'):
    df_osoby = df_osoby.sort(columns=["kadencja_ef"])
    for i in range(len(df_osoby) - 1):
        # tylko kolejne (testuje czy tak lepiej)
        if df_osoby.iloc[i + 1]["kadencja_ef"] -  df_osoby.iloc[i]["kadencja_ef"] == 0.5:
            klub_przed = "{0} ({1:.1f})".format(df_osoby.iloc[i]["klub"], df_osoby.iloc[i]["kadencja_ef"])
            klub_po = "{0} ({1:.1f})".format(df_osoby.iloc[i + 1]["klub"], df_osoby.iloc[i + 1]["kadencja_ef"])
            if (klub_przed, klub_po) not in przejscia:
                przejscia[(klub_przed, klub_po)] = 1
            else:
                przejscia[(klub_przed, klub_po)] += 1

In [12]:
df_przejscia = pd.DataFrame([{'source': k1, 'target': k2, 'value': v}
                             for (k1, k2), v in przejscia.items()])

In [13]:
df_przejscia.to_csv("przejscia.csv", index=False)

## A teraz zliczenia

In [14]:
df_kolory_linki = pd.read_csv("./partie_all.csv", index_col=0).rename(columns={"kluby": "klub"})

In [15]:
df_kolory_linki.head()

,klub,kolor,wikilink,kadencja_pierwsza,kadencja_ostatnia
1,Akcja Polska,#E9967A,/wiki/Akcja_Polska,1,1
2,Akcja Wyborcza Solidarność,#FFFF00,/wiki/Akcja_Wyborcza_Solidarno%C5%9B%C4%87,3,3
3,Alternatywa,#336699,/wiki/Alternatywa_Ruch_Spo%C5%82eczny,3,3
4,Bezpartyjny Blok Wspierania Reform,#000000,/wiki/Bezpartyjny_Blok_Wspierania_Reform,2,2
5,Biało-Czerwoni,#FF0004,/wiki/Bia%C5%82o-Czerwoni,7,7


In [16]:
df_liczebnosc = df.groupby(["klub", "kadencja_ef"]).size().reset_index().rename(columns={0: "liczba"})

In [17]:
df_liczebnosc_kolor = df_liczebnosc.merge(df_kolory_linki[["klub", "kolor", "wikilink"]], on="klub", how="left")

In [18]:
df_liczebnosc_kolor.head()

,klub,kadencja_ef,liczba,kolor,wikilink
0,Akcja Polska,1.5,3,#E9967A,/wiki/Akcja_Polska
1,Akcja Wyborcza Solidarność,3.0,201,#FFFF00,/wiki/Akcja_Wyborcza_Solidarno%C5%9B%C4%87
2,Akcja Wyborcza Solidarność,3.5,134,#FFFF00,/wiki/Akcja_Wyborcza_Solidarno%C5%9B%C4%87
3,Alternatywa,3.5,4,#336699,/wiki/Alternatywa_Ruch_Spo%C5%82eczny
4,Bezpartyjny Blok Wspierania Reform,2.0,16,#000000,/wiki/Bezpartyjny_Blok_Wspierania_Reform


In [19]:
df_liczebnosc_kolor.query("kadencja_ef == 8")

,klub,kadencja_ef,liczba,kolor,wikilink
26,Kukiz’15,8,42,NaN,NaN
38,Mniejszość Niemiecka,8,1,#0780C4,/wiki/Mniejszo%C5%9B%C4%87_Niemiecka_(komitet_...
42,Nowoczesna,8,28,NaN,NaN
56,Platforma Obywatelska,8,138,#FCA241,/wiki/Platforma_Obywatelska
75,Polskie Stronnictwo Ludowe,8,16,#1BB100,/wiki/Polskie_Stronnictwo_Ludowe
101,Prawo i Sprawiedliwość,8,235,#073A76,/wiki/Prawo_i_Sprawiedliwo%C5%9B%C4%87


In [20]:
df_liczebnosc_kolor.to_csv("liczebnosci_kadencje.csv", index=False)